# Getting Started with PyTorch


In [1]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.2.2


## Tensor Basics

Create a tensor from a Python list and convert a NumPy array to a tensor.

In [2]:
import numpy as np

# Create a tensor from a Python list
x = torch.tensor([[2.5, 0.1], [0.5, 9.6]])
print("Tensor x:\n", x)

# Convert a numpy array to a tensor
a = np.ones(5)
b = torch.from_numpy(a)
print("Tensor b:", b)

Tensor x:
 tensor([[2.5000, 0.1000],
        [0.5000, 9.6000]])
Tensor b: tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


## Autograd Example

Creating a tensor with `requires_grad=True` and compute gradients using a simple operation.

In [3]:
# Create a tensor with gradient tracking
x = torch.randn(3, requires_grad=True)
print("Initial tensor x:", x)

# Perform a simple operation (multiply by 2)
y = x * 2

# Define a vector to use in the backward pass
v = torch.tensor([1.0, 0.1, 0.001], dtype=torch.float32)

# Compute gradients (vector-Jacobian product)
y.backward(v)
print("Gradient of x:", x.grad)

Initial tensor x: tensor([ 1.0439, -0.2978, -0.7423], requires_grad=True)
Gradient of x: tensor([2.0000, 0.2000, 0.0020])


## Simple Feed-Forward Neural Network

This cell defines a basic neural network with one hidden layer and runs a dummy training loop using random data.

In [4]:
import torch.nn as nn
import torch.optim as optim

# Set device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define a simple feed-forward neural network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

# Hyperparameters
input_size = 10
hidden_size = 5
num_classes = 2
learning_rate = 0.01

# Create model instance
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Dummy training loop with random data
for epoch in range(2):  # 2 epochs for demonstration
    # Create dummy input and labels
    inputs = torch.randn(4, input_size).to(device)
    labels = torch.randint(0, num_classes, (4,)).to(device)
    
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    print(f"Epoch [{epoch+1}/2], Loss: {loss.item():.4f}")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch [1/2], Loss: 1.0792
Epoch [2/2], Loss: 0.8187


## Save and Load Model Example

This cell shows how to save the model's state and later load it into a new model instance.

In [5]:
# Save the model's state dictionary
PATH = 'model_state_dict.pth'
torch.save(model.state_dict(), PATH)
print("Model saved.")

# Create a new instance of the model and load the state dictionary
loaded_model = NeuralNet(input_size, hidden_size, num_classes)
loaded_model.load_state_dict(torch.load(PATH))
loaded_model.eval()
print("Model loaded and set to evaluation mode.")

Model saved.
Model loaded and set to evaluation mode.


# Matrix Factorization in PyTorch

In [6]:
import numpy as np
from scipy.sparse import rand as sprand
import torch

# Create synthetic ratings data
n_users = 1000
n_items = 1000

# Create a sparse random matrix with 1% density
ratings = sprand(n_users, n_items, density=0.01, format="csr")
# Replace the non-zero entries with random integer ratings from 1 to 4
ratings.data = np.random.randint(1, 5, size=ratings.nnz).astype(np.float64)
ratings = ratings.toarray()

print("Ratings shape:", ratings.shape)

Ratings shape: (1000, 1000)


In [7]:
import torch.nn as nn

class MatrixFactorization(nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super(MatrixFactorization, self).__init__()
        # User and item embeddings represent the latent factors
        self.user_factors = nn.Embedding(n_users, n_factors, sparse=True)
        self.item_factors = nn.Embedding(n_items, n_factors, sparse=True)

    def forward(self, user, item):
        # Dot product of user and item latent vectors
        return (self.user_factors(user) * self.item_factors(item)).sum(1)

# Instantiate the model
model = MatrixFactorization(n_users, n_items, n_factors=20)
print(model)

MatrixFactorization(
  (user_factors): Embedding(1000, 20, sparse=True)
  (item_factors): Embedding(1000, 20, sparse=True)
)


In [8]:
# Define the loss function and optimizer
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

# Get the indices of the non-zero ratings
rows, cols = ratings.nonzero()
# Shuffle the non-zero indices
p = np.random.permutation(len(rows))
rows, cols = rows[p], cols[p]

print(f"Total training samples: {len(rows)}")

# Training loop: iterate over each non-zero rating
for row, col in zip(rows, cols):
    optimizer.zero_grad()
    
    # Convert the rating and indices to PyTorch tensors
    rating = torch.FloatTensor([ratings[row, col]])
    user = torch.LongTensor([row])
    item = torch.LongTensor([col])
    
    # Forward pass: predict the rating
    prediction = model(user, item)
    
    # Compute the loss
    loss = loss_func(prediction, rating)
    
    # Backward pass
    loss.backward()
    
    # Update parameters
    optimizer.step()

print("Training complete.")

Total training samples: 10000
Training complete.
